In [1]:
import numpy

In [2]:
C = numpy.array([[2.33/3, 0, 0.2, 0, 0],
                 [0, 0.2, 0, 0.2, 0],
                 [0.2, 0, 0.2, 0, 0.2],
                 [0, 0.2, 0, 0.2, 0],
                 [0, 0, 0.2, 0, 0.2]])

In [3]:
C

array([[0.77666667, 0.        , 0.2       , 0.        , 0.        ],
       [0.        , 0.2       , 0.        , 0.2       , 0.        ],
       [0.2       , 0.        , 0.2       , 0.        , 0.2       ],
       [0.        , 0.2       , 0.        , 0.2       , 0.        ],
       [0.        , 0.        , 0.2       , 0.        , 0.2       ]])

In [4]:
D = numpy.array([[2.33, 0.81, 0.67, 0.92, -0.53],
                 [-0.53, 2.33, 0.81, 0.67, 0.92],
                 [0.92, -0.53, 2.33, 0.81, 0.67],
                 [0.67, 0.92, -0.53, 2.33, 0.81],
                 [0.81, 0.67, 0.92, -0.53, 2.33]])

In [5]:
b = numpy.array([4.1, 4.2, 4.2, 4.2, 4.2])

In [6]:
b = b.T
A = 3 * C - D
X = numpy.zeros(len(A))

In [7]:
def swap_columns(a, i, j):
    for k in range(len(a)):
        a[k][i], a[k][j] = a[k][j], a[k][i]

In [8]:
def check_zeros_diag(matrix, b):
    nstr = len(matrix)
    for i in range(0, nstr):
        if matrix[i][i] == 0:
            check = True
            for j in range(0, nstr):
                if matrix[i][j] != 0 and matrix[j][i] != 0:
                    swap_columns(matrix, i, j)
                    check = False
                    break
            if check:
                if b[i] == 0:
                    print('У системы бесконечное количество решений')
                else:
                    print('У системы нет решений')
                exit()

In [9]:
def straight_run(matrix, b):
    for nrow, row in enumerate(matrix):  # nrow равен номеру строки row содержит саму строку матрицы
        divider = row[nrow]  # диагональный элемент
        row /= divider  # делим на диагональный элемент.
        b[nrow] /= divider
        bfactor = b[nrow]
        # вычитаем из всех нижележащих строчек
        for lower_row in range(nrow + 1, len(matrix)):
            factor = matrix[lower_row][nrow]  # элемент строки в колонке nrow
            matrix[lower_row] -= factor * row  # вычитаем, чтобы получить ноль в колонке nrow
            b[lower_row] -= factor * bfactor
    return matrix, b

In [10]:
def reverse_run(a, b):  # перебор строк в обратном порядке
    n = len(a)
    for k in reversed(range(0, n)):
        X[k] = (b[k] - sum(a[k][i] * X[i] for i in range(k + 1, n))) / a[k][k]

In [11]:
def gauss(A, b):
    check_zeros_diag(A, b)
    straight_run(A, b)
    reverse_run(A, b)
    for i in range(len(X)):
        print("%.20f" % X[i], end=" ")
    print('\n')

In [12]:
def straight_run_column(matrix, b):
    for nrow in range(len(matrix)): # nrow равен номеру строки
        # argmax возвращает номер строки с максимальным элементом в уменьшенной матрице, которая начинается со строки
        # nrow. Поэтому нужно прибавить nrow к результату
        pivot = nrow + numpy.argmax(abs(matrix[nrow:, nrow]))
        if pivot != nrow:
            matrix[[nrow, pivot]] = matrix[[pivot, nrow]] # swap
            change = b[pivot]
            b[pivot] = b[nrow]
            b[nrow] = change
        row = matrix[nrow]
        divider = row[nrow]  # диагональный элемент
        if abs(divider) < 1e-9:
            # почти нуль на диагонали. Продолжать не имеет смысла, результат счёта неустойчив
            print(f"Результат счёта неустойчив. Максимальный элемент: {nrow}: {divider:.3g}")
            exit()
        row /= divider # делим на диагональный элемент.
        b[nrow] /= divider
        bfactor = b[nrow]
        # теперь надо вычесть приведённую строку из всех нижележащих строчек
        for lower_row in range(nrow + 1, len(matrix)):
            factor = matrix[lower_row][nrow]  # элемент строки в колонке nrow
            matrix[lower_row] -= factor * row  # вычитаем, чтобы получить ноль в колонке nrow
            b[lower_row] -= factor * bfactor

In [13]:
def gauss_maxcolumn (A, b):
    check_zeros_diag(A, b)
    straight_run_column(A,b)
    reverse_run(A,b)
    for i in range(len(X)):
        print("%.18f" % X[i], end=" ")
    print('\n')

In [14]:
def straight_run_max(matrix, b):
    for nrow in range(len(matrix)): # nrow равен номеру строки
        mrow, mcol = max_element(matrix, nrow)
        if mrow != nrow:
            matrix[[nrow, mrow]] = matrix[[mrow, nrow]] # swap
            change = b[mrow]
            b[mrow] = b[nrow]
            b[nrow] = change
        swap_columns(matrix, nrow, mcol)
        row = matrix[nrow]
        divider = row[nrow]  # диагональный элемент
        if abs(divider) < 1e-9:
            # почти нуль на диагонали. Продолжать не имеет смысла, результат счёта неустойчив
            print(f"Результат счёта неустойчив. Максимальный элемент: {divider:.3g}")
            exit()
        row /= divider # делим на диагональный элемент.
        b[nrow] /= divider
        bfactor = b[nrow]
        # теперь надо вычесть приведённую строку из всех нижележащих строчек
        for lower_row in range(nrow + 1, len(matrix)):
            factor = matrix[lower_row][nrow]  # элемент строки в колонке nrow
            matrix[lower_row] -= factor * row  # вычитаем, чтобы получить ноль в колонке nrow
            b[lower_row] -= factor * bfactor

In [15]:
def max_element(A, k):
    maximum = A[k-1][k-1]
    max_index = [k-1, k-1]
    for i in range(k-1, len(A)):
        for j in range(k-1, len(A)):
            if maximum < A[i][j]:
                maximum = A[i][j]
                max_index = [i, j]
    return max_index

In [16]:
def gauss_max(A, b):
    check_zeros_diag(A, b)
    straight_run_max(A, b)
    reverse_run(A, b)
    for i in range(len(X)):
        print("%.18f" % X[i], end=" ")
    print('\n')

In [17]:
def main():
    print("Матрица варианта 3:")
    print(A)
    
    
    n = numpy.linalg.solve(A, b)
    print("Решение с помощью нампай\n")
    for i in range(len(n)):
         print("%.20f" % n[i], end=" ")
    
    print('\nРешение с помощью собственных функций')
    print()
    gauss(A, b)

In [18]:
main()

Матрица варианта 3:
[[ 0.   -0.81 -0.07 -0.92  0.53]
 [ 0.53 -1.73 -0.81 -0.07 -0.92]
 [-0.32  0.53 -1.73 -0.81 -0.07]
 [-0.67 -0.32  0.53 -1.73 -0.81]
 [-0.81 -0.67 -0.32  0.53 -1.73]]
Решение с помощью нампай

-4.12553433769524069419 -3.04729169434607660705 -2.05524569226523379228 -1.21933703680207572440 0.69063482125132436007 
Решение с помощью собственных функций

-3.04729169434607527478 -4.12553433769523714147 -2.05524569226523379228 -1.21933703680207683462 0.69063482125132369394 



In [21]:
n = sorted(numpy.linalg.solve(A, b))
x = sorted(X)

In [19]:
def check_accuracy(w, x):
    for i in range(len(x)):
        if (abs(x[i] - w[i]) / abs(x[i])) >= 0.0001:
            return False
        else:
            print(f"|{x[i]} - {w[i]}| < 0.0001")
    return True

In [20]:
check_accuracy(n, x)

|0.6190376004729873 - 0.6190376004729873| < 0.0001
|0.6867550176536087 - 0.6867550176536087| < 0.0001
|0.8121163733241963 - 0.8121163733241963| < 0.0001
|0.8248344227063773 - 0.8248344227063773| < 0.0001
|0.8588450006050912 - 0.8588450006050912| < 0.0001


True

In [24]:
print("%.20f" % x[4], end=" ")

0.85884500060509116892 